In [39]:
import sys
import numpy as np
import cv2
import cv2.ml
from time import sleep
from datetime import datetime
import os
import numpy as np
from random import shuffle
from matplotlib import pyplot as plt
from pandas import read_csv, read_excel, DataFrame
from skimage.feature import hessian_matrix_det as Hessian
from skimage.feature import local_binary_pattern as LBP
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)

p = os.path.join('../../','raw','CookIRCamET','Images','CookTests','MedRes')
p2 = os.path.join('../../','work','CookIRCamET','Images','CookTests','MedRes')
n_components1 = 2
n_components2 = 3
n_components3 = 6

In [40]:
def localSD(mat, n):    
    mat=np.float32(mat)
    mu = cv2.blur(mat,(n,n))
    mdiff=mu-mat
    mat2=cv2.blur(np.float64(mdiff*mdiff),(n,n))
    sd = np.float32(cv2.sqrt(mat2))
    
    return sd

In [41]:
f_imgs=[]
imgs=[]
n_img=0
fs=os.listdir(p)
shuffle(fs)
for f in fs:
    if 'bgr' in f:
        f_imgs = np.append(f_imgs,f)
        bgr = cv2.imread(os.path.join(p,f),cv2.IMREAD_UNCHANGED)
        f_labels = f.split('_nofix_bgr')[0]+'_sun.tif'
        labels1 = cv2.imread(os.path.join(p,f_labels),cv2.IMREAD_UNCHANGED)
        f_labels = f.split('_nofix_bgr')[0]+'_src.tif'
        labels2 = cv2.imread(os.path.join(p,f_labels),cv2.IMREAD_UNCHANGED)
        lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
        hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
        img = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
        l,a,bb = cv2.split(lab)
        h,s,v = cv2.split(hsv)
        
        sd_l1 = localSD(l, 127)
        sd_l2 = localSD(l, 63)
        sd_l3 = localSD(l, 31)
        
        lbp_l1 = LBP(l, 32, 4, method='ror')
        lbp_l2 = LBP(l, 24, 3, method='ror')
        lbp_l3 = LBP(l, 16, 2, method='ror')
        
        sd_a1 = localSD(a, 127)
        sd_a2 = localSD(a, 63)
        sd_a3 = localSD(a, 31)
        
        lbp_a1 = LBP(a, 32, 4, method='ror')
        lbp_a2 = LBP(a, 24, 3, method='ror')
        lbp_a3 = LBP(a, 16, 2, method='ror')
        
        sd_b1 = localSD(bb, 127)
        sd_b2 = localSD(bb, 63)
        sd_b3 = localSD(bb, 31)
        
        lbp_b1 = LBP(bb, 32, 4, method='ror')
        lbp_b2 = LBP(bb, 24, 3, method='ror')
        lbp_b3 = LBP(bb, 16, 2, method='ror')
        
        sd_h1 = localSD(h, 127)
        sd_h2 = localSD(h, 63)
        sd_h3 = localSD(h, 31)
        
        lbp_h1 = LBP(h, 32, 4, method='ror')
        lbp_h2 = LBP(h, 24, 3, method='ror')
        lbp_h3 = LBP(h, 16, 2, method='ror')
        
        sd_s1 = localSD(s, 127)
        sd_s2 = localSD(s, 63)
        sd_s3 = localSD(s, 31)
        
        lbp_s1 = LBP(s, 32, 4, method='ror')
        lbp_s2 = LBP(s, 24, 3, method='ror')
        lbp_s3 = LBP(s, 16, 2, method='ror')
        
        sd_v1 = localSD(v, 127)
        sd_v2 = localSD(v, 63)
        sd_v3 = localSD(v, 31)
        
        lbp_v1 = LBP(v, 32, 4, method='ror')
        lbp_v2 = LBP(v, 24, 3, method='ror')
        lbp_v3 = LBP(v, 16, 2, method='ror')
        
        ddepth = cv2.CV_16S
        
        lap_l1 = cv2.Laplacian(l,ddepth,ksize=3)
        lap_l2 = cv2.Laplacian(l,ddepth,ksize=7)
        lap_l3 = cv2.Laplacian(l,ddepth,ksize=15)
        
        lap_a1 = cv2.Laplacian(a,ddepth,ksize=3)
        lap_a2 = cv2.Laplacian(a,ddepth,ksize=7)
        lap_a3 = cv2.Laplacian(a,ddepth,ksize=15)
        
        lap_b1 = cv2.Laplacian(bb,ddepth,ksize=3)
        lap_b2 = cv2.Laplacian(bb,ddepth,ksize=7)
        lap_b3 = cv2.Laplacian(bb,ddepth,ksize=15)
        
        lap_h1 = cv2.Laplacian(h,ddepth,ksize=3)
        lap_h2 = cv2.Laplacian(h,ddepth,ksize=7)
        lap_h3 = cv2.Laplacian(h,ddepth,ksize=15)
        
        lap_s1 = cv2.Laplacian(s,ddepth,ksize=3)
        lap_s2 = cv2.Laplacian(s,ddepth,ksize=7)
        lap_s3 = cv2.Laplacian(s,ddepth,ksize=15)
        
        lap_v1 = cv2.Laplacian(v,ddepth,ksize=3)
        lap_v2 = cv2.Laplacian(v,ddepth,ksize=7)
        lap_v3 = cv2.Laplacian(v,ddepth,ksize=15)
        
        img_size = l.shape
        bb = bb.ravel()
        a = a.ravel()
        l = l.ravel()
        h = h.ravel()
        s = s.ravel()
        v = v.ravel()
        sd_l1 = sd_l1.ravel()
        sd_l2 = sd_l2.ravel()
        sd_l3 = sd_l3.ravel()
        lbp_l1 = lbp_l1.ravel()
        lbp_l2 = lbp_l2.ravel()
        lbp_l3 = lbp_l3.ravel()
        lap_l1 = lap_l1.ravel()
        lap_l2 = lap_l2.ravel()
        lap_l3 = lap_l3.ravel()
        sd_a1 = sd_a1.ravel()
        sd_a2 = sd_a2.ravel()
        sd_a3 = sd_a3.ravel()
        lbp_a1 = lbp_a1.ravel()
        lbp_a2 = lbp_a2.ravel()
        lbp_a3 = lbp_a3.ravel()
        lap_a1 = lap_a1.ravel()
        lap_a2 = lap_a2.ravel()
        lap_a3 = lap_a3.ravel()
        sd_b1 = sd_b1.ravel()
        sd_b2 = sd_b2.ravel()
        sd_b3 = sd_b3.ravel()
        lbp_b1 = lbp_b1.ravel()
        lbp_b2 = lbp_b2.ravel()
        lbp_b3 = lbp_b3.ravel()
        lap_b1 = lap_b1.ravel()
        lap_b2 = lap_b2.ravel()
        lap_b3 = lap_b3.ravel()
        sd_h1 = sd_h1.ravel()
        sd_h2 = sd_h2.ravel()
        sd_h3 = sd_h3.ravel()
        lbp_h1 = lbp_h1.ravel()
        lbp_h2 = lbp_h2.ravel()
        lbp_h3 = lbp_h3.ravel()
        lap_h1 = lap_h1.ravel()
        lap_h2 = lap_h2.ravel()
        lap_h3 = lap_h3.ravel()
        sd_s1 = sd_s1.ravel()
        sd_s2 = sd_s2.ravel()
        sd_s3 = sd_s3.ravel()
        lbp_s1 = lbp_s1.ravel()
        lbp_s2 = lbp_s2.ravel()
        lbp_s3 = lbp_s3.ravel()
        lap_s1 = lap_s1.ravel()
        lap_s2 = lap_s2.ravel()
        lap_s3 = lap_s3.ravel()
        sd_v1 = sd_v1.ravel()
        sd_v2 = sd_v2.ravel()
        sd_v3 = sd_v3.ravel()
        lbp_v1 = lbp_v1.ravel()
        lbp_v2 = lbp_v2.ravel()
        lbp_v3 = lbp_v3.ravel()
        lap_v1 = lap_v1.ravel()
        lap_v2 = lap_v2.ravel()
        lap_v3 = lap_v3.ravel()
        feat = np.vstack((l.T,a.T,bb.T,h.T,s.T,v.T,
                          sd_l1.T,sd_l2.T,sd_l3.T,
                          lbp_l1.T,lbp_l2.T,lbp_l3.T,
                          lap_l1.T,lap_l2.T,lap_l3.T,
                          sd_a1.T,sd_a2.T,sd_a3.T,
                          lbp_a1.T,lbp_a2.T,lbp_a3.T,
                          lap_a1.T,lap_a2.T,lap_a3.T,
                          sd_b1.T,sd_b2.T,sd_b3.T,
                          lbp_b1.T,lbp_b2.T,lbp_b3.T,
                          lap_b1.T,lap_b2.T,lap_b3.T,
                          sd_h1.T,sd_h2.T,sd_h3.T,
                          lbp_h1.T,lbp_h2.T,lbp_h3.T,
                          lap_h1.T,lap_h2.T,lap_h3.T,
                          sd_s1.T,sd_s2.T,sd_s3.T,
                          lbp_s1.T,lbp_s2.T,lbp_s3.T,
                          lap_s1.T,lap_s2.T,lap_s3.T,
                          sd_v1.T,sd_v2.T,sd_v3.T,
                          lbp_v1.T,lbp_v2.T,lbp_v3.T,
                          lap_v1.T,lap_v2.T,lap_v3.T)).T
        #labels = np.sum(np.vstack((soil.ravel().T, residue.ravel().T*2, shadow.ravel().T*3, vegetation.ravel().T*4)).T,axis=1)
        labels1 = labels1.ravel()        
        labels2 = labels2.ravel() 
        #6-class
        labels3 = 3*labels1+labels2   
        
        imgs.append({'bgr':bgr,'feats':feat,'labels1':labels1,'labels2':labels2,'labels3':labels3})
        n_img=n_img+1
n_feat = feat.shape[1]

In [42]:
n_feat

60

In [43]:
feats = []
labels1 = []
labels2 = []
labels3 = []
for sample in imgs:
    feats.append(sample['feats'])
    labels1.append(sample['labels1'])
    labels2.append(sample['labels2'])
    labels3.append(sample['labels3'])

In [44]:
feats = np.array(feats).reshape((-1,n_feat)).astype(np.float32)
labels1 = np.array(labels1).reshape((-1,1)).astype(np.int32).ravel()
labels2 = np.array(labels2).reshape((-1,1)).astype(np.int32).ravel()
labels3 = np.array(labels3).reshape((-1,1)).astype(np.int32).ravel()
scaler = StandardScaler()
feats = scaler.fit_transform(feats)

In [45]:
train_feats1, test_feats1, train_labels1, test_labels1 = train_test_split(feats, labels1, test_size=0.2, random_state=42)
train_feats2, test_feats2, train_labels2, test_labels2 = train_test_split(feats, labels2, test_size=0.2, random_state=42)
train_feats3, test_feats3, train_labels3, test_labels3 = train_test_split(feats, labels3, test_size=0.2, random_state=42)

In [46]:
def cornfusion(obs,pred,nclass):
    M = np.zeros((nclass,nclass))
    for i in range(obs.shape[0]):
        o = obs[i]
        p = pred[i]
        M[o,p] = M[o,p]+1
    return M

In [47]:
#need feature selection here
svc = SGDClassifier(loss='hinge',max_iter=100000)
clf_svc1 = RFECV(svc,step=1, cv=5,n_jobs=-1)
clf_svc1.fit(train_feats1, train_labels1)

r1 = clf_svc1.ranking_
mask1 = r1==1

In [48]:
train_feats = train_feats1[:,mask1]
test_feats = test_feats1[:,mask1]

In [49]:
parameters = {'loss':('hinge', 'log_loss', 'perceptron'),'alpha':np.logspace(-6,0,7)}
svc = SGDClassifier(max_iter=100000,random_state=42)

clf_svc1 = GridSearchCV(svc, parameters,n_jobs=-1,cv=5,verbose=3)
clf_svc1.fit(train_feats, train_labels1)
clf_svc1.best_params_
model_svc1 = clf_svc1.best_estimator_


Fitting 5 folds for each of 21 candidates, totalling 105 fits


In [50]:
pred_svc1 = model_svc1.predict(test_feats)

In [51]:
import pickle
filename = 'finalized_model1_svc.pk.sav'
pickle.dump(model_svc1, open(filename, 'wb'))

In [52]:
M_svc1 = cornfusion(test_labels1,pred_svc1,n_components1)
M_svc1 = M_svc1/np.sum(np.sum(M_svc1))
recall1 = np.diag(M_svc1)/np.sum(M_svc1,axis=1)
precis1 = np.diag(M_svc1)/np.sum(M_svc1,axis=0)
M_svc1

array([[0.22505085, 0.04862707],
       [0.03594027, 0.69038182]])

In [53]:
#need feature selection here
svc = SGDClassifier(loss='hinge',max_iter=100000)
clf_svc2 = RFECV(svc,step=1, cv=5,n_jobs=-1)
clf_svc2.fit(train_feats2, train_labels2)
r2 = clf_svc2.ranking_
mask2 = r2==1

[CV 4/5] END ...........alpha=1e-06, loss=hinge;, score=0.910 total time= 4.4min
[CV 2/5] END ...........alpha=1e-06, loss=hinge;, score=0.913 total time= 4.4min
[CV 2/5] END ...........alpha=1e-05, loss=hinge;, score=0.914 total time= 2.3min
[CV 4/5] END ........alpha=0.001, loss=log_loss;, score=0.915 total time= 1.1min
[CV 4/5] END ..........alpha=0.1, loss=log_loss;, score=0.878 total time=  56.7s
[CV 5/5] END ...........alpha=1e-06, loss=hinge;, score=0.911 total time= 4.4min
[CV 4/5] END ........alpha=1e-05, loss=log_loss;, score=0.914 total time= 2.4min
[CV 5/5] END ........alpha=0.001, loss=log_loss;, score=0.915 total time=  58.7s
[CV 2/5] END ..........alpha=0.1, loss=log_loss;, score=0.879 total time= 1.1min
[CV 1/5] END ..........alpha=0.0001, loss=hinge;, score=0.915 total time=  53.3s
[CV 4/5] END ..........alpha=0.0001, loss=hinge;, score=0.915 total time=  51.1s
[CV 4/5] END .....alpha=0.0001, loss=perceptron;, score=0.832 total time=  31.3s
[CV 2/5] END ........alpha=0

In [54]:
train_feats = train_feats2[:,mask2]
test_feats = test_feats2[:,mask2]

In [55]:
parameters = {'loss':('hinge', 'log_loss', 'perceptron'),'alpha':np.logspace(-6,0,7)}
svc = SGDClassifier(max_iter=100000, random_state=42)
clf_svc2 = GridSearchCV(svc, parameters,n_jobs=-1,cv=5,verbose=3)
clf_svc2.fit(train_feats, train_labels2)

model_svc2 = clf_svc2.best_estimator_
pred_svc2 = model_svc2.predict(test_feats)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


In [56]:
M_svc2 = cornfusion(test_labels2,pred_svc2,n_components2)
M_svc2 = M_svc2/np.sum(np.sum(M_svc2))
recall2 = np.diag(M_svc2)/np.sum(M_svc2,axis=1)
precis2 = np.diag(M_svc2)/np.sum(M_svc2,axis=0)
M_svc2

array([[1.55633000e-03, 1.93195551e-02, 5.66682836e-03],
       [5.39343342e-04, 1.59362660e-01, 4.76653986e-02],
       [3.54243260e-04, 2.87075334e-02, 7.36828108e-01]])

In [57]:
precis2

array([0.63525836, 0.76842111, 0.93250455])

In [58]:
filename = 'finalized_model2_svc.pk.sav'
pickle.dump(model_svc2, open(filename, 'wb'))

In [59]:
#need feature selection here
svc = SGDClassifier(loss='hinge',max_iter=100000)
clf_svc3 = RFECV(svc,step=1, cv=5,n_jobs=-1)
clf_svc3.fit(train_feats3, train_labels3)
r3 = clf_svc3.ranking_

[CV 5/5] END ........alpha=1e-06, loss=log_loss;, score=0.896 total time= 4.5min
[CV 4/5] END ............alpha=0.01, loss=hinge;, score=0.894 total time= 1.6min
[CV 4/5] END ........alpha=0.1, loss=perceptron;, score=0.873 total time=  58.4s
[CV 3/5] END ......alpha=1e-05, loss=perceptron;, score=0.811 total time= 2.1min
[CV 1/5] END .......alpha=0.0001, loss=log_loss;, score=0.898 total time= 1.1min
[CV 4/5] END .....alpha=0.0001, loss=perceptron;, score=0.880 total time= 1.1min
[CV 1/5] END ............alpha=0.01, loss=hinge;, score=0.894 total time= 1.9min
[CV 3/5] END .............alpha=1.0, loss=hinge;, score=0.858 total time=  48.6s
[CV 2/5] END ......alpha=1e-06, loss=perceptron;, score=0.887 total time= 4.3min
[CV 4/5] END ......alpha=0.001, loss=perceptron;, score=0.862 total time=  56.4s
[CV 2/5] END .............alpha=0.1, loss=hinge;, score=0.883 total time=  59.6s
[CV 4/5] END .............alpha=1.0, loss=hinge;, score=0.858 total time=  49.3s
[CV 2/5] END ...........alph

In [60]:
mask3 = r3==1

In [61]:
train_feats = train_feats3[:,mask3]
test_feats = test_feats3[:,mask3]

In [62]:
parameters = {'loss':('hinge', 'log_loss', 'perceptron'),'alpha':np.logspace(-6,0,7)}
svc = SGDClassifier(max_iter=100000,random_state=42)
clf_svc3 = GridSearchCV(svc, parameters,n_jobs=-1,cv=5,verbose=3)
clf_svc3.fit(train_feats, train_labels3)
clf_svc3.best_params_
model_svc3 = clf_svc3.best_estimator_
#model_svc1 = svc.fit(train_feats,train_labels1)
pred_svc3 = model_svc3.predict(test_feats)

Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV 5/5] END ......alpha=1e-05, loss=perceptron;, score=0.772 total time= 6.0min
[CV 1/5] END .......alpha=0.0001, loss=log_loss;, score=0.807 total time= 6.5min
[CV 4/5] END ......alpha=0.001, loss=perceptron;, score=0.735 total time= 6.6min
[CV 5/5] END .......alpha=0.01, loss=perceptron;, score=0.752 total time= 5.1min
[CV 2/5] END ..........alpha=0.0001, loss=hinge;, score=0.791 total time= 3.3min
[CV 3/5] END ..........alpha=0.0001, loss=hinge;, score=0.791 total time= 7.7min
[CV 1/5] END ........alpha=0.001, loss=log_loss;, score=0.804 total time= 9.4min
[CV 1/5] END ........alpha=0.1, loss=perceptron;, score=0.731 total time= 3.7min
[CV 1/5] END ........alpha=1e-05, loss=log_loss;, score=0.807 total time= 6.8min
[CV 1/5] END .....alpha=0.0001, loss=perceptron;, score=0.735 total time= 4.2min
[CV 5/5] END ...........alpha=0.001, loss=hinge;, score=0.791 total time= 8.2min
[CV 1/5] END .............alpha=0.1, loss=hinge

In [63]:
M_svc3 = cornfusion(test_labels3,pred_svc3,n_components3)
M_svc3 = M_svc3/np.sum(np.sum(M_svc3))
recall3 = np.diag(M_svc3)/np.sum(M_svc3,axis=1)
precis3 = np.diag(M_svc3)/np.sum(M_svc3,axis=0)
M_svc3

array([[1.84568185e-03, 2.26375272e-03, 5.39768859e-03, 2.12758715e-06,
        2.66161152e-03, 1.73398352e-04],
       [8.13802083e-04, 2.20045701e-02, 1.59335001e-02, 2.12758715e-06,
        1.47165203e-02, 2.74033224e-03],
       [3.04244962e-04, 6.59764774e-03, 1.49064074e-01, 3.19138072e-06,
        3.82752928e-03, 4.53261166e-02],
       [1.56377655e-04, 6.59552015e-05, 8.82948666e-05, 1.39356958e-04,
        1.09996255e-02, 2.74884259e-03],
       [1.49994894e-04, 1.88504221e-03, 1.45633340e-03, 8.93586601e-05,
        1.13895059e-01, 3.38807615e-02],
       [1.26591435e-04, 4.47857094e-04, 2.32949517e-02, 4.36155365e-05,
        1.44782305e-02, 5.22375834e-01]])

In [64]:
M_svc4 = cornfusion(test_labels3,3*pred_svc1+pred_svc2,n_components3)
M_svc4 = M_svc4/np.sum(np.sum(M_svc4))
recall4 = np.diag(M_svc4)/np.sum(M_svc4,axis=1)
precis4 = np.diag(M_svc4)/np.sum(M_svc4,axis=0)
M_svc4

f3=(recall3*precis3/(recall3+precis3)*2)
f4=(recall4*precis4/(recall4+precis4)*2)

print(f3)

print(f4)

[0.23450699 0.4918559  0.74465456 0.01925055 0.73024837 0.8944697 ]
[0.18344024 0.59602612 0.76804897 0.02600248 0.75299336 0.89921019]


In [65]:
filename = 'finalized_model3_svc.pk.sav'
pickle.dump(model_svc3, open(filename, 'wb'))

In [66]:
M_svc1_df = {}
M_svc1_df['shade'] = M_svc1[:,0]
M_svc1_df['sun'] = M_svc1[:,1]
M_svc1_df = DataFrame(M_svc1_df)
M_svc1_df.to_csv(os.path.join(p2,'M1_svc.csv'))

M_svc2_df = {}
M_svc2_df['soil'] = M_svc2[:,0]
M_svc2_df['res'] = M_svc2[:,1]
M_svc2_df['can'] = M_svc2[:,2]
M_svc2_df = DataFrame(M_svc2_df)
M_svc2_df.to_csv(os.path.join(p2,'M2_svc.csv'))

M_svc3_df = {}
M_svc3_df['shade_soil'] = M_svc3[:,0]
M_svc3_df['shade_res'] = M_svc3[:,1]
M_svc3_df['shade_can'] = M_svc3[:,2]
M_svc3_df['sun_soil'] = M_svc3[:,3]
M_svc3_df['sun_res'] = M_svc3[:,4]
M_svc3_df['sun_can'] = M_svc3[:,5]
M_svc3_df = DataFrame(M_svc3_df)
M_svc3_df.to_csv(os.path.join(p2,'M3_svc.csv'))

M_svc4_df = {}
M_svc4_df['shade_soil'] = M_svc4[:,0]
M_svc4_df['shade_res'] = M_svc4[:,1]
M_svc4_df['shade_can'] = M_svc4[:,2]
M_svc4_df['sun_soil'] = M_svc4[:,3]
M_svc4_df['sun_res'] = M_svc4[:,4]
M_svc4_df['sun_can'] = M_svc4[:,5]
M_svc4_df = DataFrame(M_svc4_df)
M_svc4_df.to_csv(os.path.join(p2,'M4_svc.csv'))

In [67]:
r1

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  9,  1,  1,  1,  1,  1,  1,  2,
        1,  1, 11, 10,  1,  1,  1,  1,  1,  1,  8,  7,  1,  1,  1,  6,  1,
        1,  1,  1,  1, 13,  1,  1,  3,  1,  1,  1,  5,  4, 12,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1])

In [68]:
r2

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, 10,  1,  1,  1,  1,  1,  1,  1,
        1,  8,  1,  2,  1,  1,  1,  1,  1,  1,  1,  6,  1,  1,  1,  1,  1,
        1,  1,  7,  4,  1,  1,  1,  9,  1,  1,  1, 11,  1,  5,  1,  1,  1,
        1,  1,  1,  3,  1,  1,  1,  1,  1])

In [69]:
r3

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

[CV 4/5] END ......alpha=1e-06, loss=perceptron;, score=0.742 total time=23.7min
[CV 2/5] END ........alpha=1.0, loss=perceptron;, score=0.719 total time= 3.2min
[CV 2/5] END ...........alpha=1e-06, loss=hinge;, score=0.793 total time=27.9min
[CV 1/5] END ...........alpha=1e-06, loss=hinge;, score=0.800 total time=28.8min
